In [1]:
pip install smolagents ddgs --upgrade transformers nvidia-modelopt==0.19.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip uninstall numpy pandas scipy scikit-learn

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.12/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.12/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.12/dist-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install numpy pandas scipy scikit-learn bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from smolagents import TransformersModel, CodeAgent

### 1. Configuration for 4-bit Quantization

In [11]:
# 1. Configuration for 4-bit quantization
print("--- Loading Model: DeepSeek-R1-Distill-Qwen-7B ---")
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


--- Loading Model: DeepSeek-R1-Distill-Qwen-7B ---


In [12]:
quantization_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

### 2. Load the DeepSeek-R1 Distilled Model

In [13]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     device_map="auto", # Automatically puts layers on the GPU
#     trust_remote_code=True
# )
# model

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
# 3. Wrap the model for smolagents
# We use TransformersModel to keep everything local/offline on Kaggle
smol_model = TransformersModel(
    model_id=model_id,
    tokenizer=tokenizer,
    max_new_tokens=2048 # Math problems need space to "think"
)
smol_model

2026-01-11 09:21:32.822558: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768123292.985294     133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768123293.033366     133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768123293.417950     133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768123293.417972     133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768123293.417975     133 computation_placer.cc:177] computation placer alr

config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 86.19 MiB is free. Process 6869 has 14.65 GiB memory in use. Of the allocated memory 14.55 GiB is allocated by PyTorch, and 3.34 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### 4. Create the Agent
CodeAgent is the standard for TIR—it will write and run Python code to solve math.

In [ ]:
agent = CodeAgent(
    tools=[], # Python Interpreter is the default internal tool
    model=smol_model,
    add_base_tools=True, # Adds tools like DuckDuckGo search (if internet is on)
    max_steps=5 # Limits the agent to 5 "Think -> Code -> Result" loops per problem
)
print("--- Agent Initialized ---")

agent

### 5. Example Usage

In [ ]:
# Example usage
problem = "Find the integration of x*e^x from 0 to 1."
print(f"\nSolving: {problem}")

try:
    response = agent.run(problem)
    print(f"\n[Final Answer]: {response}")
except Exception as e:
    print(f"\n[Error]: {e}")
